In [1]:
import pandas as pd
import numpy as np

## Cirius Data (multi-seed)
- [x] load data from stain tables into excel with one sheet per-component

In [5]:
cirius_he = pd.read_excel('REFDATA/Copy of Cirius_NASH_Pilot_0_HE_Features_10805ee5-7f04-4c75-a692-d33cb0fbdc22.xlsx')
cirius_tc = pd.read_excel('REFDATA/Copy of Cirius_NASH_Pilot_0_Trichrome_Features_7420341c-91f0-46a7-a401-a493f7852c15.xlsx')

cirius_he_all = pd.read_excel('REFDATA/Cirius_NASH_Pilot_HE_Features_2020-04-22-v3_MRI.xlsx')
cirius_tc_all = pd.read_excel('REFDATA/Cirius_NASH_Pilot_Trichrome_Features_2020-04-22-v3_MRI.xlsx')


In [10]:
cirius_he_all = cirius_he_all[cirius_he_all['Not used for Analysis\n(Flag)']!='Y']
cirius_tc_all = cirius_tc_all[cirius_tc_all['Not used for Analysis\n(Flag)']!='Y']

In [11]:
cirius_he = cirius_he.merge(cirius_he_all[['SUBJID','TRT01P','Visit','H & E_ID']], 
                left_on=['PATHAI_ID'],right_on=['H & E_ID'],how='left').drop_duplicates()
cirius_tc = cirius_tc.merge(cirius_tc_all[['SUBJID','TRT01P','Visit','Trichrome_ID']],
                left_on=['PATHAI_ID'],right_on=['Trichrome_ID'],how='left').drop_duplicates()


In [12]:
cirius_tc.columns

Index(['PATHAI_ID', 'CASE_NAME', 'STAIN_NAME', 'ORIGINAL_NAME',
       'AREA (MM2) OF [ARTIFACT] IN [TISSUE]_Trichrome',
       'AREA (MM2) OF [BACKGROUND] IN [TISSUE]_Trichrome',
       'AREA (MM2) OF [USABLE TISSUE] IN [TISSUE]_Trichrome',
       'AREA (MM2) OF [FOREGROUND] IN [TISSUE]_Trichrome',
       'AREA (MM2) OF [TOTAL AREA] IN [TISSUE]_Trichrome',
       'AREA PROP [[ARTIFACT] OVER [FOREGROUND]] IN [TISSUE]_Trichrome',
       'AREA PROP [[ARTIFACT] OVER [TOTAL AREA]] IN [TISSUE]_Trichrome',
       'AREA PROP [[BACKGROUND] OVER [TOTAL AREA]] IN [TISSUE]_Trichrome',
       'AREA PROP [[USABLE TISSUE] OVER [TOTAL AREA]] IN [TISSUE]_Trichrome',
       'AREA (MM2) OF [BILE DUCT] IN [TISSUE]_TRICHROME',
       'AREA (MM2) OF [FIBROSIS] IN [TISSUE]_TRICHROME',
       'AREA (MM2) OF [TISSUE] IN [TISSUE]_TRICHROME',
       'AREA PROP [[BILE DUCT] OVER [TISSUE]] IN [TISSUE]_TRICHROME',
       'AREA PROP [[FIBROSIS] OVER [TISSUE]] IN [TISSUE]_TRICHROME',
       'AREA (MM2) OF [NORMAL FI

In [13]:
cirius_aim_cols = {
    'Fibrosis':'GNN CRN_SCORE_TRICHROME',
    'Inflammation':'GNN LOBULAR_SCORE_HE',
    'Ballooning':'GNN BALLOONING_SCORE_HE',
    'Steatosis':'GNN STEATOSIS_SCORE_HE'
}
cirius_net_path_nash_pre = {
    'Ballooning':'ballooning_score_',
    'Steatosis':'steatosis_score_',
    'Inflammation':'lobular_score_',
    'Fibrosis':'crn_score_'
}
cirius_net_paths = [
    'nashddt/partner/central/cirius',
    'nashddt/partner/index/1',
    'nashddt/partner/index/2'
]

dfs = {}
for k, aim_col in cirius_aim_cols.items():
    nps = [ cirius_net_path_nash_pre[k]+n for n in cirius_net_paths ]
    nps_rn = {n:f'NP{i}' for i, n in enumerate(nps)}
    if k == 'Fibrosis':
        dfs[k] = cirius_tc[['PATHAI_ID','CASE_NAME','SUBJID','TRT01P','Visit',cirius_aim_cols[k]]+nps].copy()
    else:
        dfs[k] = cirius_he[['PATHAI_ID','CASE_NAME','SUBJID','TRT01P','Visit',cirius_aim_cols[k]]+nps].copy()
    dfs[k].rename(columns={cirius_aim_cols[k]:'AIM'}, inplace=True)
    dfs[k].rename(columns=nps_rn, inplace = True)    

In [14]:
with pd.ExcelWriter('REFDATA/COUGAR_NP_V03.xlsx') as w:
    for k, dfc in dfs.items():
        dfc.to_excel(w, sheet_name=k, index=None)
    w.save()

# Iguana Data - single seed
- [ ] pull data from delivery table (with AIM scores)
- [ ] split into three parts: ALL, PANEL A and PANEL B

In [9]:
file = 'REFDATA/IGUANA_NP_INT200_V01.xlsx'
# core_cols = ['Subject ID','Visit','CP','PathAI slide ID','PathAI stain label']
# aim_cols = ['GNN BALLOONING_SCORE_HE','GNN LOBULAR_SCORE_HE','GNN STEATOSIS_SCORE_HE','GNN CRN_SCORE_TRICHROME']
# cp_cols = ['NASHCRN AVAL','STEATOS AVAL','NLOBI AVAL','NASHHBLN AVAL']

panel_a = ['Sanjay Kakar','Li Juan Wang','Sandy  Liu']
panel_b = ['Dhanpat Jain','Robert Najarian','Evgeny Yakirevich']

i_dfs = {}
i_dfs['Fibrosis'] = pd.read_excel(file, sheet_name='Fibrosis')
i_dfs['Inflammation'] = pd.read_excel(file, sheet_name='Inflammation')
i_dfs['Ballooning'] = pd.read_excel(file, sheet_name='Ballooning')
i_dfs['Steatosis'] = pd.read_excel(file, sheet_name='Steatosis')

for k in i_dfs.keys():
    npsa_rn = {n:f'NP_A{i}' for i, n in enumerate(panel_a)}
    npsb_rn = {n:f'NP_B{i}' for i, n in enumerate(panel_b)}
    nps_rn = {**npsa_rn, **npsb_rn}
    i_dfs[k].rename(columns=nps_rn, inplace = True)
    
with pd.ExcelWriter('REFDATA/IGUANA_NP_V01.xlsx') as w:
    for k, dfc in i_dfs.items():
        dfc.to_excel(w, sheet_name=k, index=None)
    w.save()